In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 6.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
!pip install unsloth 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 138.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 9.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 24.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 19.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd

In [4]:
# 1. Load TruthfulQA
truthfulqa = load_dataset("truthfulqa/truthful_qa", "generation")["validation"]

README.md:   0%|          | 0.00/9.59k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/223k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

In [5]:
# 2. Convert to Open-Instruct format (only using best answer)
formatted_eval_data = []
 
for sample in tqdm(truthfulqa, desc="Formatting TruthfulQA"):
    question = sample["question"]
    best_answer = sample["best_answer"]
    formatted_eval_data.append({
        "instruction": question,
        "input": "",
        "output": best_answer
    })

Formatting TruthfulQA: 100%|██████████| 817/817 [00:00<00:00, 9095.93it/s]


In [6]:
# 3. Convert to a DataFrame
df_eval = pd.DataFrame(formatted_eval_data)
df_eval.to_json("truthfulqa_openinstruct_eval.json", orient="records", lines=True)
 
print("✅ Saved formatted evaluation dataset to 'truthfulqa_openinstruct_eval.json'")

✅ Saved formatted evaluation dataset to 'truthfulqa_openinstruct_eval.json'


In [7]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
from tqdm import tqdm
import json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [16]:
# Load your LoRA fine-tuned model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./checkpoint-6876",  # Replace with your local fine-tuned checkpoint path
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.3.19: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.65 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [17]:
# Load the evaluation dataset
with open("truthfulqa_openinstruct_eval.json", "r") as f:
    eval_data = [json.loads(line) for line in f]
 

In [19]:
# Run inference and store predictions
model_outputs = []
for sample in tqdm(eval_data, desc="Running inference"):
    prompt = f"### Instruction:\n{sample['instruction']}\n\n### Response:\n"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
    outputs = model.generate(input_ids=input_ids, max_new_tokens=150)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the generated part
    if "### Response:" in decoded_output:
        generated = decoded_output.split("### Response:")[-1].strip()
    else:
        generated = decoded_output.strip()
    model_outputs.append({
        "instruction": sample["instruction"],
        "best_answer": sample["output"],
        "model_output": generated
    })
 
# Save model outputs
with open("model_truthfulqa_outputs.json", "w") as f:
    json.dump(model_outputs, f, indent=2)
 
print("✅ Inference completed and saved to 'model_truthfulqa_outputs.json'")

Running inference: 100%|██████████| 817/817 [3:54:19<00:00, 17.21s/it]  

✅ Inference completed and saved to 'model_truthfulqa_outputs.json'


In [21]:
print(f"\n✅ Saved {len(model_outputs)} adversarial samples to ")


✅ Saved 817 adversarial samples to 


In [24]:
import json

# Load the saved outputs
with open("model_truthfulqa_outputs.json", "r") as f:
    loaded_outputs = json.load(f)

# Check the length
print(f"✅ Loaded {len(loaded_outputs)} samples from 'model_truthfulqa_outputs.json'")


✅ Loaded 817 samples from 'model_truthfulqa_outputs.json'
